In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 5.2 MB/s 
     |████████████████████████████████| 5.8 MB 31.5 MB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 182 kB 60.4 MB/s 
     |████████████████████████████████| 7.6 MB 68.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=c5e854e8ba2739dc732146f1c4ae6ea52443d3a8d3f44c9a2d4edede5fd0cb17
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
!nvidia-smi 

Mon Dec  5 08:37:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    26W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import seaborn as sns
import numpy as np
from textblob import TextBlob
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/115 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

In [ ]:
pd.describe_option('display')


display.chop_threshold : float or None
    if set to a float value, all float values smaller then the given threshold
    will be displayed as exactly 0 by repr and friends.
    [default: None] [currently: None]
display.colheader_justify : 'left'/'right'
    Controls the justification of column headers. used by DataFrameFormatter.
    [default: right] [currently: right]
display.column_space No description available.
    [default: 12] [currently: 12]
display.date_dayfirst : boolean
    When True, prints and parses dates with the day first, eg 20/01/2005
    [default: False] [currently: False]
display.date_yearfirst : boolean
    When True, prints and parses dates with the year first, eg 2005/01/20
    [default: False] [currently: False]
display.encoding : str/unicode
    Defaults to the detected encoding of the console.
    Specifies the encoding to be used for strings returned by to_string,
    these are generally strings meant to be displayed on the console.
    [default: UTF-8] [curr

In [ ]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['LOINC_NUM'] = info['LOINC_NUM']
    meta_dict['LONG_COMMON_NAME'] = info['LONG_COMMON_NAME']
    return meta_dict
    
def search(query, top_k, index, model):
    t=time.time()
    query_vector = model.encode([query])
    top_k = index.search(query_vector, top_k)
    print('>>>> Results in Total Time: {}'.format(time.time()-t))
    top_k_ids = top_k[1].tolist()[0]
    top_k_ids = list(np.unique(top_k_ids))
    results =  [fetch_movie_info(idx) for idx in top_k_ids]
    return results

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Loinc.csv',memory_map=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99079 entries, 0 to 99078
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   LOINC_NUM                  99079 non-null  object 
 1   COMPONENT                  99079 non-null  object 
 2   PROPERTY                   99079 non-null  object 
 3   TIME_ASPCT                 99079 non-null  object 
 4   SYSTEM                     99079 non-null  object 
 5   SCALE_TYP                  99079 non-null  object 
 6   METHOD_TYP                 53182 non-null  object 
 7   CLASS                      99079 non-null  object 
 8   VersionLastChanged         99079 non-null  object 
 9   CHNG_TYPE                  99079 non-null  object 
 10  DefinitionDescription      12621 non-null  object 
 11  STATUS                     99079 non-null  object 
 12  CONSUMER_NAME              0 non-null      float64
 13  CLASSTYPE                  99079 non-null  int

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,14,15,16,17,22,27,28,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.head()

,LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,VersionLastChanged,CHNG_TYPE,...,COMMON_ORDER_RANK,COMMON_SI_TEST_RANK,HL7_ATTACHMENT_STRUCTURE,EXTERNAL_COPYRIGHT_LINK,PanelType,AskAtOrderEntry,AssociatedObservations,VersionFirstReleased,ValidHL7AttachmentRequest,DisplayName
0,100002-5,Specimen care is maintained,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
1,100003-3,Team communication is maintained throughout care,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
2,100004-1,Demonstrates knowledge of the expected psychos...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
3,100005-8,Demonstrates knowledge of nutritional manageme...,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN
4,100006-6,Demonstrates knowledge of medication management,Find,Pt,^Patient,Ord,NaN,SURVEY.PNDS,2.72,ADD,...,0,0,NaN,NaN,NaN,NaN,NaN,2.72,NaN,NaN


In [ ]:
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

In [ ]:
data.loc[data['LOINC_NUM'].isin(['2823-3','14749-6','4548-4'])]

,LOINC_NUM,COMPONENT,PROPERTY,TIME_ASPCT,SYSTEM,SCALE_TYP,METHOD_TYP,CLASS,VersionLastChanged,CHNG_TYPE,DefinitionDescription,STATUS,CONSUMER_NAME,CLASSTYPE,FORMULA,EXMPL_ANSWERS,SURVEY_QUEST_TEXT,SURVEY_QUEST_SRC,UNITSREQUIRED,RELATEDNAMES2,SHORTNAME,ORDER_OBS,HL7_FIELD_SUBFIELD_ID,EXTERNAL_COPYRIGHT_NOTICE,EXAMPLE_UNITS,LONG_COMMON_NAME,EXAMPLE_UCUM_UNITS,STATUS_REASON,STATUS_TEXT,CHANGE_REASON_PUBLIC,COMMON_TEST_RANK,COMMON_ORDER_RANK,COMMON_SI_TEST_RANK,HL7_ATTACHMENT_STRUCTURE,EXTERNAL_COPYRIGHT_LINK,PanelType,AskAtOrderEntry,AssociatedObservations,VersionFirstReleased,ValidHL7AttachmentRequest,DisplayName
6230,14749-6,Glucose,SCnc,Pt,Ser/Plas,Qn,NaN,CHEM,2.73,MIN,NaN,ACTIVE,NaN,1,NaN,NaN,NaN,NaN,Y,Chemistry; Endocrine; Endocrinology; Glu; Gluc; Glucoseur; Level; Pl; Plasma; Plsm; Point in time; QNT; Quan; Quant; Quantitative; Random; SerP; SerPl; SerPlas; Serum; Serum or plasma; SR; Substance concentration,Glucose SerPl-sCnc,Both,NaN,NaN,mmol/L,Glucose [Moles/volume] in Serum or Plasma,mmol/L,NaN,NaN,NaN,4506,0,0,NaN,NaN,NaN,NaN,NaN,1.0k,NaN,Glucose [Moles/Vol]
20814,2823-3,Potassium,SCnc,Pt,Ser/Plas,Qn,NaN,CHEM,2.73,MIN,NaN,ACTIVE,NaN,1,NaN,NaN,NaN,NaN,Y,Chemistry; k; K+; Level; Pl; Plasma; Plsm; Point in time; Potass; QNT; Quan; Quant; Quantitative; Random; SerP; SerPl; SerPlas; Serum; Serum or plasma; SR; Substance concentration; UniversalLabOrders,Potassium SerPl-sCnc,Both,NaN,NaN,mmol/L,Potassium [Moles/volume] in Serum or Plasma,mmol/L,NaN,NaN,NaN,7,74,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,Potassium [Moles/Vol]
39751,4548-4,Hemoglobin A1c/Hemoglobin.total,MFr,Pt,Bld,Qn,NaN,CHEM,2.73,MIN,NaN,ACTIVE,NaN,1,NaN,NaN,NaN,NaN,Y,Blood; Chemistry; Endocrine; Endocrinology; GHb; Glycated hemoglobin; Glycohaemogolobin; Glycohemogolobin; Glycosylated hemoglobin; HA1c; Haemoglobin; Hb; HbA1c; Hgb; Hgb A1c; Mass fraction; Percent; Point in time; QNT; Quan; Quant; Quantitative; Random; Tot; Totl; WB; Whole blood,HbA1c MFr Bld,Both,NaN,NaN,%,Hemoglobin A1c/Hemoglobin.total in Blood,%,NaN,NaN,Release 2.70: CLASS: Changed Class from HEM/BC to CHEM as A1C tests are more correctly classified in CHEM.; Previous Releases: Changing from the property from Mass fraction to substance fraction has not effect on the meaning when the molecular weight of the numerator and the denominator are the same (for practical purposes). These terms had had a property of MFR in the past and we changed to SFR because we thought the LOINC community preferred SFR for HbA1c. But we mis understood the preference is to use the MFR variant- so we have changed back.,88,48,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,HbA1c (Bld) [Mass fraction]


In [ ]:
import gc
df = data[['LOINC_NUM', 'LONG_COMMON_NAME','COMPONENT','SHORTNAME','RELATEDNAMES2','EXAMPLE_UNITS','EXAMPLE_UCUM_UNITS']]


In [ ]:
df['LOINC_NUM']

NameError: ignored

In [ ]:
df['LONG_COMMON_NAME'] = df['LONG_COMMON_NAME'].astype(str)

<ipython-input-13-29fc3b1508df>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['LONG_COMMON_NAME'] = df['LONG_COMMON_NAME'].astype(str)


In [ ]:
df['RELATEDNAMES2'] = df['RELATEDNAMES2'].astype(str)

<ipython-input-14-4db57c77596b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RELATEDNAMES2'] = df['RELATEDNAMES2'].astype(str)


In [ ]:
df['SHORTNAME'] = df['SHORTNAME'].astype(str)

<ipython-input-15-d35d83a9f413>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SHORTNAME'] = df['SHORTNAME'].astype(str)


In [ ]:
df['RELATEDNAMES2'] = df['RELATEDNAMES2'].astype(str)

<ipython-input-16-4db57c77596b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RELATEDNAMES2'] = df['RELATEDNAMES2'].astype(str)


In [ ]:
df['EXAMPLE_UNITS'] = df['EXAMPLE_UNITS'].astype(str)

<ipython-input-17-9e00404c05cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EXAMPLE_UNITS'] = df['EXAMPLE_UNITS'].astype(str)


In [ ]:
df['EXAMPLE_UCUM_UNITS'] = df['EXAMPLE_UCUM_UNITS'].astype(str)

<ipython-input-18-44d03bf10e27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EXAMPLE_UCUM_UNITS'] = df['EXAMPLE_UCUM_UNITS'].astype(str)


In [ ]:
type = []
for index, row in df.iterrows():
  if isinstance(row['LONG_COMMON_NAME'], str) and isinstance(row['EXAMPLE_UNITS'], str) and isinstance(row['EXAMPLE_UCUM_UNITS'], str) and isinstance(row['RELATEDNAMES2'], str) and isinstance(row['SHORTNAME'], str):
    type.append(row['RELATEDNAMES2']+" "+ row['LONG_COMMON_NAME']+" "+row['SHORTNAME']+" "+row['EXAMPLE_UNITS']+" "+row['EXAMPLE_UCUM_UNITS'])
  else:
    type.append('None')

print(len(type))

print(len(type), df.shape)

nump_a = np.array(type)
df['name'] = nump_a.tolist()
df

99079
99079 (99079, 7)


<ipython-input-19-60067b207fa2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'] = nump_a.tolist()


In [ ]:
df.dropna(inplace=True)
df.drop_duplicates(subset=['Plot'],inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 68 kB/s 


In [ ]:
import faiss
encoded_data = model.encode(df.name.tolist())
encoded_data = np.asarray(encoded_data.astype('float32'))
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, len(df))))
faiss.write_index(index, 'loinc.index')

TSH IU/ml , Random Blood Sugar, Triglyceride lipid profile serum R mg/dL T-CO2 Venous  mmol/L

In [ ]:
from pprint import pprint

query="RANDOM BLOOD GLUCOSE blood mmol/L"
results=search(query, top_k=15, index=index, model=model)

print("\n")
for result in results:
    print('\t',pprint(result))

>>>> Results in Total Time: 0.13087749481201172


{'LOINC_NUM': '12219-2',
 'LONG_COMMON_NAME': 'Glucose [Mass/volume] in Peritoneal fluid --4 hours post '
                     'peritoneal dialysis'}
	 None
{'LOINC_NUM': '12220-0',
 'LONG_COMMON_NAME': 'Glucose [Mass/volume] in Peritoneal fluid --12 hours '
                     'post peritoneal dialysis'}
	 None
{'LOINC_NUM': '1538-8',
 'LONG_COMMON_NAME': 'Glucose [Mass/volume] in Urine --4 hours post 100 g '
                     'glucose PO'}
	 None
{'LOINC_NUM': '1541-2',
 'LONG_COMMON_NAME': 'Glucose [Mass/volume] in Urine --5 hours post 100 g '
                     'glucose PO'}
	 None
{'LOINC_NUM': '1545-3',
 'LONG_COMMON_NAME': 'Deprecated Glucose [Mass/volume] in Serum or Plasma --6 '
                     'hours post 100 g glucose PO'}
	 None
{'LOINC_NUM': '1546-1',
 'LONG_COMMON_NAME': 'Glucose [Mass/volume] in Urine --6 hours post 100 g '
                     'glucose PO'}
	 None
{'LOINC_NUM': '27872-1',
 'LONG_COMMON_NAME': 

# Re-Ranker: Cross-Encoder

The retriever has to be efficient for large document collections with millions of entries. However, it might return irrelevant candidates.

A re-ranker based on a Cross-Encoder can substantially improve the final results for the user. The query and a possible document is passed simultaneously to transformer network, which then outputs a single score between 0 and 1 indicating how relevant the document is for the given query.

The advantage of Cross-Encoders is the higher performance, as they perform attention across the query and the document.

Scoring thousands or millions of (query, document)-pairs would be rather slow. Hence, we use the retriever to create a set of e.g. 100 possible candidates which are then re-ranked by the Cross-Encoder.


In [ ]:
## Load our cross-encoder. Use fast tokenizer to speed up the tokenization
from sentence_transformers import CrossEncoder
cross_model = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-6', max_length=512)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/541 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:

def cross_score(model_inputs):
    scores = cross_model.predict(model_inputs)
    return scores
# query = 'cbc'
model_inputs = [[query,item['LONG_COMMON_NAME']] for item in results]
scores = cross_score(model_inputs)
#Sort the scores in decreasing order
ranked_results = [{'LONG_COMMON_NAME': inp['LONG_COMMON_NAME'], 'Score': score} for inp, score in zip(results, scores)]
ranked_results = sorted(ranked_results, key=lambda x: x['Score'], reverse=True)


In [ ]:
print("\n")
for result in ranked_results:
    print('\t',pprint(result))



{'LONG_COMMON_NAME': 'Triglyceride in Lipoprotein a [Mass/volume] in Serum or '
                     'Plasma',
 'Score': 0.07290701}
	 None
{'LONG_COMMON_NAME': 'Triglyceride [Mass/volume] in Serum or Plasma --12 hours '
                     'fasting',
 'Score': 0.00516791}
	 None
{'LONG_COMMON_NAME': '21-Deoxycortisol [Moles/volume] in Serum or Plasma --30 '
                     'minutes post 250 ug corticotropin',
 'Score': 0.00057176704}
	 None
{'LONG_COMMON_NAME': '25-hydroxyvitamin D3 [Moles/volume] in Serum or Plasma '
                     '--1 hour post dose calcium',
 'Score': 0.0005617461}
	 None
{'LONG_COMMON_NAME': 'Androstenedione [Mass/volume] in Serum or Plasma --30 '
                     'minutes post 250 ug corticotropin IM',
 'Score': 0.00035143743}
	 None
{'LONG_COMMON_NAME': 'Dehydroepiandrosterone (DHEA) [Mass/volume] in Serum or '
                     'Plasma --15 minutes post 250 ug corticotropin',
 'Score': 0.00032073766}
	 None
{'LONG_COMMON_NAME': 'Dehydroepi

In [ ]:
!pip install bert-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.2 MB/s 


In [ ]:

# check your installation
import bert_score
bert_score.__version__

'0.3.12'

In [ ]:
from bert_score import score

In [ ]:
ref=["Lymphocyte %"]

In [ ]:
ranked_results_bert = []

for cand in results:
    P, R, F1 = score([cand['LONG_COMMON_NAME']], ref, lang='en')
    ranked_results_bert.append({'Title': cand['LONG_COMMON_NAME'], 'Score': F1.numpy()[0]})
    

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

In [ ]:
#Sort the scores in decreasing order
ranked_results_bert = sorted(ranked_results_bert, key=lambda x: x['Score'], reverse=True)
print("\n")
for result in ranked_results_bert:
    print('\t',pprint(result))

In [ ]:
final_results = pd.DataFrame()
final_results['faiss_ranking'] = [item['Title'] for item in results]
final_results['cross_encoder'] = [item['Title'] for item in ranked_results]
final_results['bert_score'] = [item['Title'] for item in ranked_results_bert]


In [ ]:
final_results.head()

In [ ]:

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
item_i = model.encode(['action'])
item_i = np.asarray(item_i.astype('float32'))

item_j = model.encode(['drama'])
item_j = np.asarray(item_j.astype('float32'))

item_k = model.encode(['romance'])
item_k = np.asarray(item_k.astype('float32'))

In [ ]:
data = np.asarray([item_i, item_j, item_k])

In [ ]:
data = data.reshape((3,768))

In [ ]:
tsne = TSNE(n_components=2)
tsne_results = tsne.fit_transform(data)

In [ ]:
tsne_results.shape

In [ ]:
pca_df = pd.DataFrame()
pca_df['x'] = tsne_results[:,0]
pca_df['y'] = tsne_results[:,1]

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(
    x="x", y="y",
    hue=['action','drama','romance'],
    palette=sns.color_palette("hls", 3),
    data=pca_df,
    legend="full",
    alpha=0.8
)

In [ ]:
# generate random integer values
from random import seed
from random import randint
# seed random number generator
seed(1)

action_movie_scores=[]
drama_movie_scores=[]
romance_movie_scores=[]
# generate some integers
for _ in range(10):
    action_movie_score = randint(0, 10)
    drama_movie_score = randint(0, 10)
    romance_movie_score = randint(0, 10)
    action_movie_scores.append(action_movie_score)
    drama_movie_scores.append(drama_movie_score)
    romance_movie_scores.append(romance_movie_score)

In [ ]:
user_watch_hist = pd.DataFrame()
user_watch_hist['action_movie'] = action_movie_scores
user_watch_hist['drama_movie'] = drama_movie_scores
user_watch_hist['romance_movie'] = romance_movie_scores

In [ ]:

user_watch_hist.plot(kind='bar',figsize=(16,10))

In [ ]:
user_watch_hist

In [ ]:
import scipy.stats as stats

In [ ]:
action_movie_zscore =   stats.zscore(user_watch_hist['action_movie'])[-1]
drama_movie_zscore = stats.zscore(user_watch_hist['drama_movie'])[-1]
romance_movie_zscore = stats.zscore(user_watch_hist['romance_movie'])[-1]

In [ ]:
def weight(i):
    alpha = 0.5
    return alpha*pow((1-alpha),10-i)

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
x = np.asarray([action_movie_zscore, drama_movie_zscore, romance_movie_zscore])
genre_weights = softmax(x)

In [ ]:
user_encoded_vector = np.asarray([item_i*genre_weights[0]+ item_j*genre_weights[1] +item_k*genre_weights[2]])
user_encoded_vector = user_encoded_vector.reshape((1,768))

In [ ]:
from pprint import pprint

query="Artificial Intelligence based action movie"
results=search(query, top_k=5, index=index, model=model)


print("\n")
for result in results:
    print('\t',pprint(result))

In [ ]:
candidate_plots = [x['Plot'] for x in results]

In [ ]:
from sentence_transformers import SentenceTransformer, util

#Compute embeddings
embeddings = model.encode(candidate_plots)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(user_encoded_vector, embeddings)

#Find the pairs with the highest cosine similarity scores
titles = [x['Title'] for x in results]

ranked_user_behaviour = [{'Title':x ,'Score': y} for x,y in zip(titles,cosine_scores.numpy()[0])]
ranked_user_behaviour = sorted(ranked_user_behaviour, key=lambda x: x['Score'], reverse=True)

In [ ]:
final_results = pd.DataFrame()
final_results['faiss_ranking'] = [item['Title'] for item in results]
final_results['cross_encoder'] = [item['Title'] for item in ranked_results]
final_results['bert_score'] = [item['Title'] for item in ranked_results_bert]
final_results['user_interaction_ranking'] = [item['Title'] for item in ranked_user_behaviour]

In [ ]:
final_results

In [ ]:
def fetch_movie_info(dataframe_idx):
    info = df.iloc[dataframe_idx]
    meta_dict = {}
    meta_dict['Title'] = info['Title']
    return meta_dict
    

t=time.time()
query_vector = user_encoded_vector
top_k = index.search(query_vector, 20)
print('>>>> Recommendation Results in Total Time: {}'.format(time.time()-t))
top_k_ids = top_k[1].tolist()[0]
top_k_ids = list(np.unique(top_k_ids))
[fetch_movie_info(idx) for idx in top_k_ids]
